<h1>Algorithm</h1>

=> A bigram language model is trained from the corpus. <br />
=> Another bigram language model is trained after reversing the sentences from the corpus. <br />
=> Given the context word, the model randomly decides to add a prefix or a suffix.<br />
=> To add prefix, we generate the word from the language model trained of reversed sentences.<br />
=> To add suffix, we generate the word from the language model trained on original sentences.<br />

<h1>Code</h1>

In [1]:
import os
import nltk
import string
import random

from nltk.util import ngrams

In [45]:
FILE_PATH = './corpus.txt'

stopwords = ['', '(', ')', '{', '}', '\\', '--', ':', '-', "'s"]
punc = string.punctuation + "``" + "''" + '"'

In [46]:
with open(FILE_PATH, 'r') as f:
    data = f.read().lower().replace('\n',' ')

In [47]:
len(data)

3954021

In [48]:
type(data)

str

In [49]:
def tokenized_words(data):
    sents = []
    for sent in nltk.sent_tokenize(data):
        words = [word for word in nltk.word_tokenize(sent) if word not in stopwords and word not in punc]
        sents.append(words)
        
    return sents

In [50]:
def tokenized_rev_words(data):
    sents = []
    for sent in nltk.sent_tokenize(data):
        words = [word for word in nltk.word_tokenize(sent) if word not in stopwords and word not in punc]
        sents.append(list(reversed(words)))
        
    return sents

In [51]:
sents = tokenized_words(data)
rev_sents = tokenized_rev_words(data)

In [52]:
len(sents)

40992

In [11]:
train_corpus = [word for sent in sents for word in sent]
rev_train_corpus = [word for sent in rev_sents for word in sent] 

In [12]:
len(train_corpus)

666665

In [21]:
def ngram_freq_dist(corpus, ngram=1):
    if isinstance(corpus, list) and len(corpus)>0:
        train_corpus=corpus
    elif type(corpus) is str:
        train_corpus=nltk.word_tokenize(corpus)
    else:
        print('Error')
        return None
    
    freq_dist=None
    if ngram==1:
        freq_dist = nltk.FreqDist(train_corpus) #freq distibution for unigrams
    elif ngram==2:
        freq_dist = nltk.ConditionalFreqDist(nltk.ngrams(train_corpus, 2))# conditional freq dist for bigrams
    elif ngram==3:
        trigrams_as_bigrams=[]
        trigram =[a for a in ngrams(train_corpus, 3)]
        trigrams_as_bigrams.extend([((t[0],t[1]), t[2]) for t in trigram])
        freq_dist = nltk.ConditionalFreqDist(trigrams_as_bigrams)# conditional freq dist for trigrams
    else:
        print('Supported upto trigrams only')
    return freq_dist

In [23]:
cfd_2gram = ngram_freq_dist(train_corpus, 2)
cfd_2gram_rev = ngram_freq_dist(rev_train_corpus, 2)

cpd_2gram = nltk.ConditionalProbDist(cfd_2gram, nltk.MLEProbDist)
cpd_2gram_rev = nltk.ConditionalProbDist(cfd_2gram_rev, nltk.MLEProbDist)

In [53]:
def generate_txt_bigram_model_random(cprob_2gram, cprob_2gram_rev, initialword, numwords=15):
    text = initialword
    suf_word = initialword
    pre_word = initialword
    for index in range(numwords):
        if random.random() > 0.5:
            try:
                suf_word = cprob_2gram[suf_word].generate()
                text = text + " " + suf_word
            except Exception as e:
                print('Can not generate the sentence')
                return
        else:
            try: 
                pre_word = cprob_2gram_rev[pre_word].generate()
                text = pre_word + ' ' + text
            except Exception as e:
                print('Can not generate the sentence')
                return
    return text

<h1>Results</h1>

In [56]:
for _ in range(5):
    print(generate_txt_bigram_model_random(cpd_2gram, cpd_2gram_rev, 'education'))
    print('----------------------------------------')

afford college and national guidelines highlight how education we do right to balance we are underinsured
----------------------------------------
are with the france attack on our strength and education w… rt thebriefing2016 but somehow changing
----------------------------------------
the international brotherhood of higher education can take courageous leader we have to tour of health
----------------------------------------
today we unanimously approved our education should live presidential campaign … when people of eximbankus has
----------------------------------------
nation as mayors must pursue higher education amp across the battle to go vote … this
----------------------------------------
